In [4]:
import requests
import pandas as pd

def get_enrolled_users(course_id):
    """Lấy danh sách user từ Moodle"""
    base_url = "http://139.99.103.223:9090/webservice/rest/server.php"
    params = {
        "wstoken": "becf3b3a2a6f06858f2f16505719f5ef",
        "wsfunction": "core_enrol_get_enrolled_users",
        "moodlewsrestformat": "json",
        "courseid": course_id
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Chỉ lấy id và username
    users = [{"id": user["id"], "username": user["username"]} for user in data]
    return pd.DataFrame(users)


def add_id_to_csv(course_id, csv_path, output_path):
    """Ghép cột id vào file CSV dựa theo username"""
    # Lấy dữ liệu từ Moodle
    df_api = get_enrolled_users(course_id)

    # Đọc file CSV gốc
    df_csv = pd.read_csv(csv_path)

    # Ghép (merge) 2 bảng dựa trên username
    df_merged = pd.merge(df_csv, df_api, on="username", how="left")

    # Đưa cột id lên đầu
    cols = ['id'] + [col for col in df_merged.columns if col != 'id']
    df_merged = df_merged[cols]

    # Lưu lại file mới
    df_merged.to_csv(output_path, index=False)
    print(f"✅ Đã thêm cột id và lưu file tại: {output_path}")

    return df_merged


# 👉 Gọi hàm
updated_df = add_id_to_csv(
    course_id=2,
    csv_path="../data/users_have_col.csv",
    output_path="../data/users_have_col_with_id.csv"
)

# In ra vài dòng đầu kiểm tra
print(updated_df.head())

✅ Đã thêm cột id và lưu file tại: ../data/users_have_col_with_id.csv
   id      username password                   email firstname  lastname group
0   5  student10000  pass123  student10000@email.com   Student     10000   khá
1   6  student10001  pass123  student10001@email.com   Student     10001   yếu
2   7  student10002  pass123  student10002@email.com   Student     10002   khá
3   8  student10003  pass123  student10003@email.com   Student     10003   yếu
4   9  student10004  pass123  student10004@email.com   Student     10004   khá


In [6]:
import pandas as pd
import random

# Đọc dữ liệu
df_users = pd.read_csv("../data/users_have_col_with_id.csv")
df_grades = pd.read_csv("../data/simulate_grade.csv")

# Hàm tạo điểm random theo group
def generate_random_grade(group):
    group = str(group).strip().lower()
    if group == "giỏi":
        # Học sinh giỏi đôi khi cũng chỉ được 7.8 hoặc 7.9
        return round(random.uniform(7.5, 9.8), 1)
    elif group == "khá":
        return round(random.uniform(6.0, 8.0), 1)
    elif group == "yếu":
        return round(random.uniform(3.0, 6.0), 1)
    else:
        # Nếu không xác định được nhóm
        return round(random.uniform(4.0, 6.5), 1)

# Ghép dữ liệu user vào simulate_grade dựa trên id
df_merged = pd.merge(df_grades, df_users[['id', 'group']], left_on='user_id', right_on='id', how='left')

# Sinh điểm ngẫu nhiên ban đầu
df_merged['finalgrade'] = df_merged['group'].apply(generate_random_grade)

# Điều chỉnh điểm theo độ khó bài thi
def adjust_grade(row):
    grade = row['finalgrade']
    item = str(row.get('grade_item', '')).lower()

    if "easy" in item:
        grade += 0.5
    elif "hard" in item:
        grade -= 0.5

    # Giới hạn trong [0, 10]
    grade = max(0, min(10, grade))
    return round(grade, 1)

df_merged['finalgrade'] = df_merged.apply(adjust_grade, axis=1)

# ✅ Xóa cột id dư an toàn
df_merged = df_merged.drop(columns=[col for col in ['id', 'id_y'] if col in df_merged.columns])


# Xuất file kết quả
df_merged.to_csv("simulate_grade_with_score.csv", index=False)
print("✅ Đã tạo file simulate_grade_with_score.csv với logic khó/dễ và giới hạn điểm [0,10]")

# In thử vài dòng đầu
print(df_merged.head())

✅ Đã tạo file simulate_grade_with_score.csv với logic khó/dễ và giới hạn điểm [0,10]
   id_x  user_id  courseid                   grade_item itemtype  finalgrade  \
0   379        5         2                          NaN   course         7.5   
1   378        5         2    bài kiểm tra bài 1 - easy      mod         7.6   
2   386        5         2    bài kiểm tra bài 1 - hard      mod         7.0   
3   385        5         2  bài kiểm tra bài 1 - medium      mod         7.5   
4   388        5         2    bài kiểm tra bài 2 - easy      mod         7.8   

   timemodified group  
0             0   khá  
1             0   khá  
2             0   khá  
3             0   khá  
4             0   khá  
